In [1]:
import pdfplumber

In [2]:
text_chunks = []
chunk_size = 200

In [3]:
with pdfplumber.open('made-with-cc.pdf') as pdf:
    for page in pdf.pages[62:162]:
        text = page.extract_text()
        words = text.split()
        for i in range(0,len(words),chunk_size):
            chunk = " ".join(words[i:i + chunk_size])
            text_chunks.append(chunk)

In [4]:
import os
import pinecone
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
PINECONE_API = os.getenv('PINECONE_API')
OPENAI_API = os.getenv('OPENAI_API_KEY')

In [7]:
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec

In [8]:
pc =Pinecone(api_key=PINECONE_API)

In [9]:
spec = ServerlessSpec(cloud='aws',region='us-east-1')

In [10]:
index_name = 'made-with-cc'

In [11]:
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

In [12]:
dimension = 384
pc.create_index(
    name=index_name,
    dimension=dimension,
    metric = 'cosine',
    spec = spec
)

In [13]:
index = pc.Index(index_name)

In [14]:
from sentence_transformers import SentenceTransformer

In [15]:
model = SentenceTransformer('all-MiniLM-L6-v2')

C:\Users\harsh\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
def embed_text(text):
    return model.encode(text)

In [17]:
for i, chunk in enumerate(text_chunks):
    embedding = embed_text(chunk)
    index.upsert(vectors=[(f"chunk_{i}", embedding.tolist(), {"text": chunk})])

C:\Users\harsh\AppData\Roaming\Python\Python312\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [28]:
def search_DB(query):
    query_embedding = embed_text(query)
    result = index.query(
        vector=query_embedding.tolist(),
        top_k=5,
        include_metadata=True
    )
    return result['matches']

OPENAI's API

In [29]:
import openai

In [31]:
def answer(query,contexts):
    context = " ".join(chunk['metadata']['text']for chunk in contexts)
    prompt = f"Based on the information from 'Made with Creative Commons':{context} \n answer the question: {query}'"
    response = openai.Completion.create(
        model='gpt-4',
        prompt=prompt,
        max_tokens = 200
    )
    return response.choices[0].text.strip()

In [32]:
def gen_ans(query):
    matches = search_DB(query)
    answer = answer(query,matches)
    return answer

In [ ]:
user_query = "What role do Creative Commons licenses play in open education resources"
answer = gen_ans(user_query)
print(f"Answer:\n{answer}")